In [82]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch.nn.functional as F
from datasets import Dataset as HFDataset
import torch.optim as optim
from tqdm import tqdm

In [26]:
CSV_FILE = Path("Annotate _ Datasets - Annotation v3 FINAL NA BEH.csv")
TEXT_COLUMN = "Text"
LABEL_COLUMNS = [
    "Age",
    "Gender",
    "Physical",
    "Race",
    "Religion",
    "Politics",
    "Others",
]
MAX_FEATS = 5_000
NGRAM_RANGE = (1, 2)
UNCERTAINTY_THRESHOLD = 0.05

In [27]:
df = pd.read_csv(CSV_FILE, encoding='latin1')
df = df.drop(columns= ['Unnamed: 8','Unnamed: 9', 'Unnamed: 10'], axis= 1)
df

,Text,Age,Gender,Physical,Race,Religion,Politics,Others
0,Tangina Chinese. Pinapasakop naman tayo ng isa...,0,0,0,1,0,1,0
1,ay bakit?porke ba nasa America ka ay di ka na ...,0,0,0,1,0,0,0
2,Ano ba kasing pinaglalaban ng mga Iglesia sa E...,0,0,0,0,1,0,0
3,Anong nililipad mo dyan PUTANGINA KANG SIRAULO...,0,0,1,0,0,0,0
4,The neighbor's smell got in the condo pota amo...,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...
4338,Ang bagal ng serbisyo kasi puro matatanda ang ...,1,0,0,0,0,0,0
4339,"Wala nang naitulong 'yang mga gurang na 'yan, ...",1,0,0,0,0,0,0
4340,"Mga Gen Z, puro lang salita, wala namang gawa.",1,0,0,0,0,0,0
4341,Sana mawala na sa sistema 'yang mga tanda na '...,1,0,0,0,0,0,0


In [28]:
# Stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

tagalog_stopwords = set(["akin","aking","ako","alin","am","amin","aming","ang","ano","anumang","apat","at","atin","ating","ay","bababa","bago","bakit","bawat","bilang","dahil","dalawa","dapat","din","dito","doon","gagawin","gayunman","ginagawa","ginawa","ginawang","gumawa","gusto","habang","hanggang","hindi","huwag","iba","ibaba","ibabaw","ibig","ikaw","ilagay","ilalim","ilan","inyong","isa","isang","itaas","ito","iyo","iyon","iyong","ka","kahit","kailangan","kailanman","kami","kanila","kanilang","kanino","kanya","kanyang","kapag","kapwa","karamihan","katiyakan","katulad","kaya","kaysa","ko","kong","kulang","kumuha","kung","laban","lahat","lamang","likod","lima","maaari","maaaring","maging","mahusay","makita","marami","marapat","masyado","may","mayroon","mga","minsan","mismo","mula","muli","na","nabanggit","naging","nagkaroon","nais","nakita","namin","napaka","narito","nasaan","ng","ngayon","ni","nila","nilang","nito","niya","niyang","noon","o","pa","paano","pababa","paggawa","pagitan","pagkakaroon","pagkatapos","palabas","pamamagitan","panahon","pangalawa","para","paraan","pareho","pataas","pero","pumunta","pumupunta","sa","saan","sabi","sabihin","sarili","sila","sino","siya","tatlo","tayo","tulad","tungkol","una","walang"])
ALL_STOPWORDS= STOPWORDS.union(tagalog_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# Lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')
ENG_LEMMA = WordNetLemmatizer()

def load_lemma_dict(file_path):
    lemma_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if ':' in line:
                word, lemma = line.strip().split(':')
                lemma_dict[word.strip()] = lemma.strip()
    return lemma_dict

def lemmatize_text(word, lemma_dict):
    return lemma_dict.get(word, word)

TAGALOG_LEMMA= load_lemma_dict('filipino_lemma.txt')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
def clean_text(text: str) -> str:
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    # Remove stopwords
    tokens = text.split()
    tokens= [t for t in tokens if t not in ALL_STOPWORDS]
    # Lemmatization
    lemmatized_tokens = []
    for t in tokens:
        lemma = TAGALOG_LEMMA.get(t)
        if lemma is not None:
            lemmatized_tokens.append(lemma)
        else:
            lemmatized_tokens.append(ENG_LEMMA.lemmatize(t))
    return " ".join(lemmatized_tokens)

def tokenize(text: str) -> List[str]:
    return re.findall(r"\b\w+\b", text.lower())


In [31]:
# df = pd.read_csv(CSV_FILE)
df["clean_text"] = df[TEXT_COLUMN].apply(clean_text)
df["tokens"] = df["clean_text"].apply(tokenize)
df["token_text"] = df["tokens"].apply(lambda toks: " ".join(toks))

# 60/30/10
X_temp, X_val_original, y_temp, y_val = train_test_split(
    df["token_text"], df[LABEL_COLUMNS], test_size=0.10, random_state=42
)

X_train_original, X_test_original, y_train, y_test = train_test_split(
    X_temp, y_temp, test_size=0.3333, random_state=42
)

In [32]:
df['Text']

0       Tangina Chinese. Pinapasakop naman tayo ng isa...
1       ay bakit?porke ba nasa America ka ay di ka na ...
2       Ano ba kasing pinaglalaban ng mga Iglesia sa E...
3       Anong nililipad mo dyan PUTANGINA KANG SIRAULO...
4       The neighbor's smell got in the condo pota amo...
                              ...                        
4338    Ang bagal ng serbisyo kasi puro matatanda ang ...
4339    Wala nang naitulong 'yang mga gurang na 'yan, ...
4340       Mga Gen Z, puro lang salita, wala namang gawa.
4341    Sana mawala na sa sistema 'yang mga tanda na '...
4342    Hindi na nakakasabay sa uso ang mga matatanda,...
Name: Text, Length: 4343, dtype: object

In [72]:
df

,Text,Age,Gender,Physical,Race,Religion,Politics,Others,clean_text,tokens,token_text
0,Tangina Chinese. Pinapasakop naman tayo ng isa...,0,0,0,1,0,1,0,tangina chinese pinapasakop naman tangatangang...,"[tangina, chinese, pinapasakop, naman, tangata...",tangina chinese pinapasakop naman tangatangang...
1,ay bakit?porke ba nasa America ka ay di ka na ...,0,0,0,1,0,0,0,bakitporke ba nasa america di pinoyulol magpab...,"[bakitporke, ba, nasa, america, di, pinoyulol,...",bakitporke ba nasa america di pinoyulol magpab...
2,Ano ba kasing pinaglalaban ng mga Iglesia sa E...,0,0,0,0,1,0,0,ba kasi laban iglesia edsa gulo nga dun dagan ...,"[ba, kasi, laban, iglesia, edsa, gulo, nga, du...",ba kasi laban iglesia edsa gulo nga dun dagan ...
3,Anong nililipad mo dyan PUTANGINA KANG SIRAULO...,0,0,1,0,0,0,0,ano nililipad mo dyan putangina kang siraulo k...,"[ano, nililipad, mo, dyan, putangina, kang, si...",ano nililipad mo dyan putangina kang siraulo k...
4,The neighbor's smell got in the condo pota amo...,0,0,1,1,0,0,0,neighbor smell got condo pota amoy bumbay,"[neighbor, smell, got, condo, pota, amoy, bumbay]",neighbor smell got condo pota amoy bumbay
...,...,...,...,...,...,...,...,...,...,...,...
4338,Ang bagal ng serbisyo kasi puro matatanda ang ...,1,0,0,0,0,0,0,bagal serbisyo kasi puro tanda tatrabaho,"[bagal, serbisyo, kasi, puro, tanda, tatrabaho]",bagal serbisyo kasi puro tanda tatrabaho
4339,"Wala nang naitulong 'yang mga gurang na 'yan, ...",1,0,0,0,0,0,0,wala nang itulo yang gurang yan puro sakit lang,"[wala, nang, itulo, yang, gurang, yan, puro, s...",wala nang itulo yang gurang yan puro sakit lang
4340,"Mga Gen Z, puro lang salita, wala namang gawa.",1,0,0,0,0,0,0,gen z puro lang salita wala nama gawa,"[gen, z, puro, lang, salita, wala, nama, gawa]",gen z puro lang salita wala nama gawa
4341,Sana mawala na sa sistema 'yang mga tanda na '...,1,0,0,0,0,0,0,sana wala sistema yang tanda yan bago,"[sana, wala, sistema, yang, tanda, yan, bago]",sana wala sistema yang tanda yan bago


In [34]:
vectorizer = TfidfVectorizer(
    tokenizer=tokenize,
    max_features=MAX_FEATS,
    ngram_range=NGRAM_RANGE,
    lowercase=True,
)

X_train_tfidf = vectorizer.fit_transform(X_train_original).astype(np.float32).toarray()


c:\Users\Admin\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
from sklearn.preprocessing import normalize
X_test = vectorizer.transform(X_test_original).astype(np.float32).toarray()
X_val = vectorizer.transform(X_val_original).astype(np.float32).toarray()

# normalize the features
X_train = normalize(X_train_tfidf, norm='l2')
X_test = normalize(X_test, norm='l2')
X_val = normalize(X_val, norm='l2')

In [36]:
class LinearSVM:
    def __init__(self, n_features: int, C: float = 10.0, lr: float = 0.1, epochs: int = 150, seed: int = 42):
        self.C = C
        self.lr0 = lr
        self.epochs = epochs
        self.rng = np.random.default_rng(seed)
        self.w = np.zeros(n_features, dtype=np.float32)
        self.b = 0.0

    def fit(self, X: np.ndarray, y: np.ndarray):
        y_signed = np.where(y > 0, 1.0, -1.0).astype(np.float32)
        n_samples = X.shape[0]
        for epoch in range(1, self.epochs + 1):
            indices = self.rng.permutation(n_samples)
            eta = self.lr0 / (1 + epoch * 0.01)
            for i in indices:
                xi = X[i]
                yi = y_signed[i]
                margin = yi * (np.dot(self.w, xi) + self.b)
                if margin < 1:
                    self.w += eta * self.C * yi * xi  # removed decay
                    self.b += eta * self.C * yi

        # bias centering fix
        self.b = -np.mean(X @ self.w)

    def decision_function(self, X: np.ndarray) -> np.ndarray:
        return X @ self.w + self.b

    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.decision_function(X) >= 0).astype(np.int8)


class OneVsRestSVM:
    def __init__(self, n_labels: int, n_features: int, **svm_kwargs):
        self.n_labels = n_labels
        self.models = [LinearSVM(n_features, **svm_kwargs) for _ in range(n_labels)]

    def fit(self, X: np.ndarray, Y: np.ndarray):
        for idx in range(self.n_labels):
            self.models[idx].fit(X, Y[:, idx])

    def decision_function(self, X: np.ndarray) -> np.ndarray:
        return np.column_stack([m.decision_function(X) for m in self.models])

    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.decision_function(X) >= 0).astype(np.int8)


In [37]:
svm = OneVsRestSVM(
    n_labels=len(LABEL_COLUMNS),
    n_features=X_train_tfidf.shape[1],
    C=1.0,
    lr=0.01,
    epochs=5,
)
svm.fit(X_train, y_train.values.astype(np.int8))

In [38]:
RULE_LEXICONS = {
    "Age": ["tanda", "boomer", "bata", "matanda", "gurang", "genz", "millenial"],
    "Gender": ["bakla", "tomboy", "babae kasi", "bading", "gay", "lesbian"],
    "Physical": ["pangit", "pandak", "tabatsoy", "baldado"],
    "Race": ["chinese", "unggoy", "nigger", "chingchong"],
    "Religion": ["iglesia", "muslim", "terrorist", "jihad", "infidel", "catholic", "pari"],
    "Politics": ["bbm", "dilawan", "kakampink", "komunista", "presidente", "lugaw"],
    "Others": ["tanga", "ulol", "gago", "bobo", "stupido", "putangina mo", "putangina", "tangina"],
}


def rule_based_labels(cleaned_text: str, lexicons: dict, label_order: List[str]) -> List[int]:
    labels = [0] * len(label_order)
    for i, label in enumerate(label_order):
        for kw in lexicons[label]:
            if kw in cleaned_text:
                labels[i] = 1
                break
    return labels

In [51]:
def hybrid_predict(
    raw_texts,
    vec,
    model,
    lexicons,
    label_order,
    thr: float = UNCERTAINTY_THRESHOLD,
):
    feats = vec.transform(raw_texts).astype(np.float32).toarray()
    decisions = model.decision_function(feats)
    ml_preds = (decisions >= 0).astype(np.int8)

    final_preds = []
    for idx, text in enumerate(raw_texts):
        cleaned = clean_text(text)
        rule_labels = rule_based_labels(cleaned, lexicons, label_order)
        combined = []
        for j in range(len(label_order)):
            if abs(decisions[idx, j]) < thr:
                combined.append(rule_labels[j])
            else:
                combined.append(int(ml_preds[idx, j]))
        final_preds.append(combined)
    return np.array(final_preds, dtype=np.int8)


In [46]:
def evaluate(y_true, y_pred, label_names):
    print("=== Classification Report ===")
    print(
        classification_report(
            y_true,
            y_pred,
            target_names=label_names,
            zero_division=0,
        )
    )
    print("Hamming Loss:", hamming_loss(y_true, y_pred))
    print("Subset Accuracy:", accuracy_score(y_true, y_pred))


In [52]:
# concatenate the original Series before vectorization
X_combined_original = pd.concat([X_test_original, X_val_original])
y_combined = pd.concat([y_test, y_val])

# transform the combined original Series using the fitted vectorizer
X_combined_tfidf = vectorizer.transform(X_combined_original).astype(np.float32).toarray()

# normalize
X_combined = normalize(X_combined_tfidf, norm='l2')

y_combined_pred = hybrid_predict(X_combined_original.tolist(), vectorizer, svm, RULE_LEXICONS, LABEL_COLUMNS)
print(" SVM-RULE BASED HYBRID PERFORMANCE ")
evaluate(y_combined.values, y_combined_pred, LABEL_COLUMNS)

 SVM-RULE BASED HYBRID PERFORMANCE 
=== Classification Report ===
              precision    recall  f1-score   support

         Age       0.85      0.94      0.90       304
      Gender       0.74      0.89      0.80       281
    Physical       0.59      0.65      0.61       293
        Race       0.88      0.79      0.83       201
    Religion       0.92      0.86      0.89       290
    Politics       0.90      0.86      0.88       227
      Others       0.35      0.82      0.49       310

   micro avg       0.66      0.83      0.74      1906
   macro avg       0.75      0.83      0.77      1906
weighted avg       0.73      0.83      0.76      1906
 samples avg       0.71      0.84      0.75      1906

Hamming Loss: 0.09247082031892158
Subset Accuracy: 0.5385500575373993


In [42]:
#test
svm1 = OneVsRestSVM(len(LABEL_COLUMNS), X_train.shape[1], C=0.1, lr=0.01, epochs=1)
svm2 = OneVsRestSVM(len(LABEL_COLUMNS), X_train.shape[1], C=100.0, lr=10.0, epochs=50)

svm1.fit(X_train, y_train.values.astype(np.int8))
svm2.fit(X_train, y_train.values.astype(np.int8))

dec1 = svm1.decision_function(X_train[:10])
dec2 = svm2.decision_function(X_train[:10])

print("Difference in decision outputs:", np.abs(dec1 - dec2).sum())


Difference in decision outputs: 42666.24


In [43]:
# pure model-only prediction
y_pred = svm.predict(X_combined)
evaluate(y_combined.values, y_pred, LABEL_COLUMNS)


=== Classification Report ===
              precision    recall  f1-score   support

         Age       0.70      0.96      0.81       304
      Gender       0.65      0.96      0.78       281
    Physical       0.41      0.85      0.55       293
        Race       0.43      0.94      0.59       201
    Religion       0.88      0.92      0.90       290
    Politics       0.65      0.89      0.75       227
      Others       0.43      0.88      0.58       310

   micro avg       0.56      0.92      0.69      1906
   macro avg       0.59      0.92      0.71      1906
weighted avg       0.60      0.92      0.71      1906
 samples avg       0.65      0.92      0.73      1906

Hamming Loss: 0.1274864376130199
Subset Accuracy: 0.39298043728423476


In [44]:
def count_rule_usage(raw_texts, model, vectorizer, label_order, thr=0.2):
    feats = vectorizer.transform(raw_texts).astype(np.float32).toarray()
    decisions = model.decision_function(feats)
    rule_used = (np.abs(decisions) < thr).astype(np.int8)
    percent_used = rule_used.mean(axis=0) * 100
    return dict(zip(label_order, percent_used))

print("Rule override % per label:")
print(count_rule_usage(X_combined_original.tolist(), svm, vectorizer, LABEL_COLUMNS))


Rule override % per label:
{'Age': 88.3199079401611, 'Gender': 89.93095512082854, 'Physical': 95.56962025316456, 'Race': 99.30955120828538, 'Religion': 88.60759493670885, 'Politics': 90.73647871116226, 'Others': 99.88492520138091}


In [61]:
# Custom SGD Classifier with regularization for comparison
class SGD:
    def __init__(self, n_features: int, C: float = 10.0, lr: float = 0.1, epochs: int = 150, seed: int = 42):
        self.C = C
        self.lr0 = lr
        self.epochs = epochs
        self.rng = np.random.default_rng(seed)
        self.w = np.zeros(n_features, dtype=np.float32)
        self.b = 0.0
    
    def fit(self, X: np.ndarray, y: np.ndarray):
        y_signed = np.where(y > 0, 1.0, -1.0).astype(np.float32)
        n_samples = X.shape[0]
        for epoch in range(1, self.epochs + 1):
            indices = self.rng.permutation(n_samples)
            eta = self.lr0 / (1 + epoch * 0.01)
            for i in indices:
                xi = X[i]
                yi = y_signed[i]
                margin = yi * (np.dot(self.w, xi) + self.b)
                
                # L2 regularization term
                reg_term = eta / (self.C * n_samples)
                
                if margin < 1:
                    # Apply regularization and hinge loss gradient
                    self.w = (1 - reg_term) * self.w + eta * self.C * yi * xi
                    self.b += eta * self.C * yi
                else:
                    # Only apply regularization
                    self.w = (1 - reg_term) * self.w
        
        # bias centering fix
        self.b = -np.mean(X @ self.w)

    def decision_function(self, X: np.ndarray) -> np.ndarray:
        return X @ self.w + self.b
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.decision_function(X) >= 0).astype(np.int8)

class OneVsRestSGD:
    def __init__(self, n_labels: int, n_features: int, **sgd_kwargs):
        self.n_labels = n_labels
        self.models = [SGD(n_features, **sgd_kwargs) for _ in range(n_labels)]

    def fit(self, X: np.ndarray, Y: np.ndarray):
        for idx in range(self.n_labels):
            self.models[idx].fit(X, Y[:, idx])

    def decision_function(self, X: np.ndarray) -> np.ndarray:
        return np.column_stack([m.decision_function(X) for m in self.models])

    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.decision_function(X) >= 0).astype(np.int8)

In [62]:
sdg= OneVsRestSGD(
    n_labels= len(LABEL_COLUMNS),
    n_features= X_train_tfidf.shape[1],
    C=1.0,
    lr=0.01,
    epochs=5
)
sdg.fit(X_train, y_train.values.astype(np.int8))


In [65]:
y_combined_pred = hybrid_predict(X_combined_original.tolist(), vectorizer, sdg, RULE_LEXICONS, LABEL_COLUMNS)

print(" SDG-RULE BASED HYBRID PERFORMANCE ")
evaluate(y_combined.values, y_combined_pred, LABEL_COLUMNS)

 SDG-RULE BASED HYBRID PERFORMANCE 
=== Classification Report ===
              precision    recall  f1-score   support

         Age       0.85      0.94      0.90       304
      Gender       0.74      0.89      0.81       281
    Physical       0.59      0.64      0.61       293
        Race       0.88      0.78      0.83       201
    Religion       0.92      0.87      0.89       290
    Politics       0.91      0.86      0.89       227
      Others       0.35      0.81      0.49       310

   micro avg       0.66      0.83      0.74      1906
   macro avg       0.75      0.83      0.77      1906
weighted avg       0.73      0.83      0.76      1906
 samples avg       0.71      0.84      0.75      1906

Hamming Loss: 0.0927996054578333
Subset Accuracy: 0.5368239355581128


In [56]:
# pure model-only prediction
y_pred = sdg.predict(X_combined)
evaluate(y_combined.values, y_pred, LABEL_COLUMNS)

=== Classification Report ===
              precision    recall  f1-score   support

         Age       0.69      0.96      0.80       304
      Gender       0.65      0.97      0.78       281
    Physical       0.41      0.85      0.55       293
        Race       0.43      0.94      0.59       201
    Religion       0.88      0.92      0.90       290
    Politics       0.65      0.89      0.75       227
      Others       0.44      0.88      0.58       310

   micro avg       0.56      0.92      0.69      1906
   macro avg       0.59      0.92      0.71      1906
weighted avg       0.60      0.92      0.71      1906
 samples avg       0.65      0.92      0.73      1906

Hamming Loss: 0.12715765247410818
Subset Accuracy: 0.3947065592635213


In [79]:
#MBERT Performance
# Load pre-trained mBERT model and tokenizer
# 60/30/10
X_temp, X_val_original, y_temp, y_val = train_test_split(
    df["clean_text"], df[LABEL_COLUMNS], test_size=0.10, random_state=42
)

X_train_original, X_test_original, y_train, y_test = train_test_split(
    X_temp, y_temp, test_size=0.3333, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def tokenize(example):
    return tokenizer(example["clean_text"], padding="max_length", truncation=True, max_length=128)

train_hf = HFDataset.from_pandas(X_train_original.to_frame(name='clean_text'))
val_hf = HFDataset.from_pandas(X_val_original.to_frame(name='clean_text'))
test_hf = HFDataset.from_pandas(X_test_original.to_frame(name='clean_text'))


train_enc = train_hf.map(tokenize)
val_enc = val_hf.map(tokenize)
test_enc = test_hf.map(tokenize)

def add_labels(encodings, labels):
    encodings_with_labels = []
    for i, encoding in enumerate(encodings):
        encoding_copy = dict(encoding)
        encoding_copy['labels'] = labels[i].tolist()
        encodings_with_labels.append(encoding_copy)
    return encodings_with_labels

train_enc_with_labels = add_labels(list(train_enc), y_train.values)
val_enc_with_labels = add_labels(list(val_enc), y_val.values)
test_enc_with_labels = add_labels(list(test_enc), y_test.values)

for name, dset in zip(
    ["train_enc", "val_enc", "test_enc"],
    [train_enc, val_enc, test_enc]
):
    cols_to_remove = []
    if "clean_text" in dset.column_names:
        cols_to_remove.append("clean_text")
    if "__index_level_0__" in dset.column_names:
        cols_to_remove.append("__index_level_0__")
    if cols_to_remove:
        dset = dset.remove_columns(cols_to_remove)
    locals()[name] = dset  # update the variable in the local scope

Map: 100%|██████████| 1303/1303 [00:00<00:00, 1924.23 examples/s]


In [86]:
# PyTorch dataset
class HateSpeechTorchDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings[idx]["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(self.encodings[idx]["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(self.encodings[idx]["labels"], dtype=torch.float)
        }

train_dataset = HateSpeechTorchDataset(train_enc_with_labels)
val_dataset = HateSpeechTorchDataset(val_enc_with_labels)
test_dataset = HateSpeechTorchDataset(test_enc_with_labels)



In [87]:
class mBERTHateSpeechClassifier(nn.Module):
    def __init__(self, n_labels, dropout=0.1):
        super(mBERTHateSpeechClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("bert-base-multilingual-uncased")
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_labels)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [88]:
def train_model(model, train_loader, val_loader, device, epochs=3, lr=2e-5):
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    # Learning rate scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        total_train_loss = 0
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        for batch in train_pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            total_train_loss += loss.item()
            train_pbar.set_postfix({'loss': loss.item()})
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        total_val_loss = 0
        
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]')
        with torch.no_grad():
            for batch in val_pbar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)
                
                total_val_loss += loss.item()
                val_pbar.set_postfix({'loss': loss.item()})
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Average training loss: {avg_train_loss:.4f}')
        print(f'  Average validation loss: {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_mbert_model.pth')
            print(f'  New best model saved!')
        
        print()
    
    return train_losses, val_losses

In [89]:
# Evaluation function
def evaluate_model(model, test_loader, device, threshold=0.5):
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            probabilities = torch.sigmoid(logits)
            predictions = (probabilities > threshold).float()
            
            all_predictions.append(predictions.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_probabilities.append(probabilities.cpu().numpy())
    
    predictions = np.vstack(all_predictions)
    labels = np.vstack(all_labels)
    probabilities = np.vstack(all_probabilities)
    
    return predictions, labels, probabilities

In [90]:
def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Get number of labels from your data
    n_labels = y_train.shape[1]  # Assuming y_train is your multilabel target
    
    # Create data loaders
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    model = mBERTHateSpeechClassifier(n_labels=n_labels)
    model.to(device)
    
    print(f'Model loaded with {n_labels} labels')
    print(f'Total parameters: {sum(p.numel() for p in model.parameters()):,}')
    
    # Train model
    print('Starting training...')
    train_losses, val_losses = train_model(
        model, train_loader, val_loader, device, 
        epochs=3, lr=2e-5
    )
    
    # Load best model
    model.load_state_dict(torch.load('best_mbert_model.pth'))
    
    # Evaluate on test set
    print('Evaluating on test set...')
    predictions, labels, probabilities = evaluate_model(model, test_loader, device)
    
    # Calculate metrics
    exact_match = accuracy_score(labels, predictions)
    hamming = hamming_loss(labels, predictions)
    
    print(f'Test Results:')
    print(f'Exact Match Ratio: {exact_match:.4f}')
    print(f'Hamming Loss: {hamming:.4f}')
    
    # Per-label classification report
    label_names = [f'Label_{i}' for i in range(n_labcudaels)]  # Replace with actual label names if available
    
    for i in range(n_labels):
        print(f'\nLabel {i} ({label_names[i]}):')
        report = classification_report(
            labels[:, i], predictions[:, i], 
            target_names=['Not Present', 'Present'],
            digits=4
        )
        print(report)
    
    return model, predictions, probabilities, train_losses, val_losses

# Run the training
if __name__ == "__main__":
    model, predictions, probabilities, train_losses, val_losses = main()


Using device: cpu
Model loaded with 7 labels
Total parameters: 167,361,799
Starting training...


Epoch 1/3 [Train]:   2%|▏         | 3/163 [04:47<4:15:21, 95.76s/it, loss=0.662] 


KeyboardInterrupt: 